In [1]:
!pip install transformers[torch]
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 4.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import GPT2LMHeadModel, GPT2Tokenizer, TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments
import torch


In [3]:
df = pd.read_json('/content/drive/MyDrive/Colab Notebooks/news_generation/data_ktmpost.json')
df.head()

,title,article,image_url,category,link,name
0,Nepal lose to West Indies A by 76 runs,West Indies A defeated Nepal by 76 runs in the...,https://assets-api.kathmandupost.com/thumb.php...,sports,https://kathmandupost.com/cricket/2024/05/01/n...,sports_0
1,Nepal win fifth T20 to salvage some pride,"Aasif Sheikh, Anil Sah and Kushal Malla all st...",https://assets-api.kathmandupost.com/thumb.php...,sports,https://kathmandupost.com/cricket/2024/05/04/n...,sports_1
2,"APF, Sudurpaschim keep winning run in Lalitpur...",Defending champions Armed Police Force Club ma...,https://assets-api.kathmandupost.com/thumb.php...,sports,https://kathmandupost.com/cricket/2024/05/13/a...,sports_2
3,West Indies A defeat Nepal by 10 runs,Nepal suffered a 10-run defeat at the hands of...,https://assets-api.kathmandupost.com/thumb.php...,sports,https://kathmandupost.com/cricket/2024/04/28/w...,sports_3
4,Kritika gives Mayor’s XI flying start,Kritika Marasini took four wickets as Lalitpur...,https://assets-api.kathmandupost.com/thumb.php...,sports,https://kathmandupost.com/sports/2024/05/12/kr...,sports_4


In [4]:
titles = df['title'].tolist()
articles = df['article'].tolist()

In [5]:
with open('/content/drive/MyDrive/Colab Notebooks/news_generation/train_data.txt', 'w', encoding='utf-8') as f:
    for title, article in zip(titles, articles):
        f.write(f"Title: {title}\nArticle: {article}\n\n")


In [ ]:
model_name = "gpt2"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
def load_dataset(file_path, tokenizer):
    return TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=128
    )

train_dataset = load_dataset('/content/drive/MyDrive/Colab Notebooks/news_generation/train_data.txt', tokenizer)


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
)


In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/Colab Notebooks/news_generation/results',
    overwrite_output_dir=True,
    num_train_epochs=100,
    per_device_train_batch_size=20,
    save_steps=10_000,
    save_total_limit=2,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
)


In [ ]:
trainer.train()

In [ ]:
# Save the model and tokenizer
model.save_pretrained('/content/drive/MyDrive/Colab Notebooks/news_generation/saved_model')
tokenizer.save_pretrained('/content/drive/MyDrive/Colab Notebooks/news_generation/saved_model')

In [ ]:
# Load the model and tokenizer
model = GPT2LMHeadModel.from_pretrained('/content/drive/MyDrive/Colab Notebooks/news_generation/saved_model')
tokenizer = GPT2Tokenizer.from_pretrained('/content/drive/MyDrive/Colab Notebooks/news_generation/saved_model')

In [ ]:
# Generate Article
def generate_article(title, model, tokenizer, max_length=512):
    input_text = f"Title: {title}\nArticle:"
    inputs = tokenizer.encode(input_text, return_tensors='pt')
    outputs = model.generate(inputs, max_length=max_length, num_return_sequences=1)
    article = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return article.replace(input_text, '').strip()

In [ ]:
# Example title to generate article
new_title = "Nepal lost to west indies by 60 runs"
generated_article = generate_article(new_title, model, tokenizer)
print(generated_article)